In [2]:
import time
import pymongo
from pymongo import MongoClient

start = time.time()

#--------------------FUNCTIONS FOR LARGE PROTEIN FASTA ASSEMBLY USING MONGODB--------------------

#Extract proteinIDs from DTASelect Proteins for mongoDB protein fasta assembly
#input: DTAselect Protein file
#output: proteinID list
def extractProteinID(DTA_select_file):
    proteinlist1 = []
    with open(DTA_select_file, 'r') as h:
        file1 = h.readlines()
    for line1 in file1:
        line2 = line1.split('\t')
        if (line2[0] == "Unique"):
            index1 = file1.index(line1)
            break
    for line1 in range((index1+1), len(file1)):
        line2 = file1[line1].split('\t')
        if (line2[0] != "") and (line2[0] != '*') and (line2[1] != "Proteins"):
            proteinlist1.append(line2[0])
        elif (line2[1] == "Proteins"):
            break
    return proteinlist1

#search mongodatabase for protein sequences
#input: proteinID list and name of output fasta file
#output: fasta file containing proteinIDs and protein sequences
def findProtSeq(plist1, output_file):
    client = MongoClient("localhost", 27017)
    db = client['ProtDB_compil1501']
    outputfile1 = open(output_file, 'w')
    for item1 in plist1:
        protein1 = db.ProtDB_compil1501.find_one({ "b" : item1 })
        outputfile1.write(">" + protein1['d'] + "\n")
        outputfile1.write(protein1['s'] + "\n")
    outputfile1.close()

#-----------------FUNCTIONS FOR ASSEMBLING CLUSTER-SPECTRAL COUNT TABLE FOR DESEQ2-----------------

#create list of unique vs non-unique peptides; 
#input is DTASelect Peptide file
#output is printed result, no return
def determineUniqueness(file_to_open):
    unique = []
    nonunique = []
    with open(file_to_open, 'r') as f:
        file1 = f.readlines()  
    for line1 in range(1, len(file1)):
        line2 = file1[line1].split('\t')
        if '*' in line2[13]:
            unique.append(line2[12])
        else:
            nonunique.append(line2[12]) 
    print("Unique peptides:", len(set(unique)))
    print("Non-unique peptides:", len(set(nonunique)))

#create count of all CD-HIT clusterIDs
#input is CD-HIT cluster file
#output is integer count of clusters
def createClusterIndex(cluster_file):
    with open(cluster_file, 'r') as g:
        file1 = g.readlines()
    clustercount = 0
    for line1 in file1:
        if line1[0] == ">":
            clustercount += 1
    return clustercount

#create dictionary linking proteinIDs (keys) to CD-HIT clusterIDs (values)
#input is CD-HIT cluster file
#output is ProteinID-cdhitID dictionary
def createProteinClusterDict(cluster_file):
    with open(cluster_file, 'r') as g:
        file1 = g.readlines()
    clusterID = ""
    proteindict1 = {}
    for line1 in file1:
        line2 = line1.split()
        if ">" in line2[0][0]:
            clusterID = line2[1]
        else:
            proteindict1[line2[2][1:-3]] = clusterID
    return proteindict1

#create file linking proteinIDs to peptides in DTAselect file 
#first input is DTASelect Protein file
#first output is file (dump.txt) where clusterIDs(non-indented) associate with peptides (indented)
#create dictionary linking peptides (keys; str) and clusterIDs (values); clusterIDs = list of non-redundant integers
#second input is protein-clusterID dictionary
#second output is peptide-ClusterID dictionary
def createPeptideClusterIDDict(DTA_select_file, proteindict1):
    #----------PART 1----------
    with open(DTA_select_file, 'r') as h:
        file1 = h.readlines()
    for line1 in file1:
        line2 = line1.split('\t')
        if (line2[0] == "Unique"):
            index1 = file1.index(line1)
            break
    outputfile1 = open("dump.txt", 'w')
    for line1 in range((index1+1), len(file1)):
        line2 = file1[line1].split('\t')
        if (line2[0] != "") and (line2[0] != '*') and (line2[1] != "Proteins"):
            #Note this Try/Except line was added to take care of contaminant proteins in non-microbiome searches
            try:
                outputfile1.write(proteindict1[line2[0]] + "\n")
            except:
                pass
        elif ((line2[0] == "") or (line2[0] == '*')) and (line2[1] != "Proteins"):
            #Note splice at line2[14] added to take care of non-microbiome searches
            outputfile1.write("\t" + line2[14][1:-1] + "\n")
        else:
            break
    outputfile1.close()
    #----------PART 2----------
    with open("dump.txt", 'r') as i:
        file2 = i.readlines()
    clusterlist1 = []
    peptidedict1 = {}
    newcluster = False
    for line1 in file2:
        line2 = line1.split('\t')
        if (line2[0] != ''):
            if newcluster == True:
                clusterlist1 = []
                newcluster = False
            clusterlist1.append(int(line2[0].replace('\n', '')))
        elif (line2[0] == ''):
            peptidedict1[line2[1].replace('\n', '')] = clusterlist1
            newcluster = True
        else:
            break
    for key1 in peptidedict1.keys():
        peptidedict1[key1] = list(set(peptidedict1[key1]))
    return peptidedict1

#create dictionary linking peptides (keys; str) to spec counts (values; int)
#input is DTASelect Peptide file
#output is peptide-SC dictionary
def createPeptideSCDict(file_to_open):
    with open(file_to_open, 'r') as j:
        file1 = j.readlines()
    speccountdict1 = {}
    for line1 in range(1, len(file1)):
        line2 = file1[line1].split('\t')
        if line2[12] in speccountdict1:
            speccountdict1[line2[12][1:-1].replace('\n', '')] += int(line2[11])
        else:
            speccountdict1[line2[12][1:-1].replace('\n', '')] = int(line2[11])
    return speccountdict1

#create dictionary linking clusterIDs (keys; int) to spec counts (values; int)
#peptides mapping to >1 cluster are not counted, empty clusters are not filled in with 0
#input is peptide-SC dictionary and peptide-ClusterID dictionary
#output is clusterID-SC dictionary
def createClusterIDSCDict(SCdict1, clustdict1):
    clusterspeccountdict1 = {}
    tempclusternumber1 = 0
    multiclustercount = 0
    singleclustercount = 0
    for key1 in clustdict1.keys():
        if len(clustdict1[key1]) == 1:
            if clustdict1[key1][0] in clusterspeccountdict1:
                clusterspeccountdict1[clustdict1[key1][0]] += SCdict1[key1]
            else:
                clusterspeccountdict1[clustdict1[key1][0]] = SCdict1[key1]
            singleclustercount += 1
        else:
            #for item1 in clustdict1[key1]:
            #    clusterspeccountdict1[item1] = 0
            multiclustercount += 1
    print("Single Cluster Assignments:", singleclustercount)
    print("Multi Cluster Assignments:", multiclustercount)
    return clusterspeccountdict1

#input: cluster-spectral count dictionary, total number of clusters
#output: cluster-spectral count dictionary with all cluster keys present
def missingValues(clusterSCDict, clustercount):
    for item1 in range(0, clustercount):
        if item1 not in clusterSCDict.keys():
            clusterSCDict[item1] = 0
    return clusterSCDict

#processes all lower functions for single Compil2 Search
#input is cdhit cluster file, DTAselect protein file, and DTAselect peptide file
#output is cluster-spectral count dictionary with 0 values filled in; cluster# (keys; int) vs SC (values; int)
def processAll(cdhit_file, DTAselect_protein_file, DTAselect_peptide_file):
    var1 = createProteinClusterDict(cdhit_file)
    var2 = createPeptideClusterIDDict(DTAselect_protein_file, var1)
    var3 = createPeptideSCDict(DTAselect_peptide_file)
    var4 = createClusterIDSCDict(var3, var2)
    var5 = createClusterIndex(cdhit_file)
    var6 = missingValues(var4, var5)
    return var6

end = time.time()
print(end-start)

0.002170085906982422


In [5]:
start = time.time()

#-----------Extract ProteinIDs-----------

list1 = extractProteinID("MB-DTASelect/1-MB.txt")
list2 = extractProteinID("MB-DTASelect/2-MB.txt")
list3 = extractProteinID("MB-DTASelect/3-MB.txt")
list4 = extractProteinID("MB-DTASelect/4-MB.txt")
list5 = extractProteinID("MB-DTASelect/5-MB.txt")
list6 = extractProteinID("MB-DTASelect/6-MB.txt")
list7 = extractProteinID("MB-DTASelect/7-MB.txt")
list8 = extractProteinID("MB-DTASelect/8-MB.txt")
list9 = extractProteinID("MB-DTASelect/9-MB.txt")
list10 = extractProteinID("MB-DTASelect/10-MB.txt")
list11 = extractProteinID("MB-DTASelect/11-MB.txt")
list12 = extractProteinID("MB-DTASelect/12-MB.txt")
list13 = extractProteinID("MB-DTASelect/13-MB.txt")
list14 = extractProteinID("MB-DTASelect/14-MB.txt")
list15 = extractProteinID("MB-DTASelect/15-MB.txt")
list16 = extractProteinID("MB-DTASelect/16-MB.txt")
list17 = extractProteinID("MB-DTASelect/17-MB.txt")

#------Create Master ProteinID list------

masterset1 = set(list1).union(set(list2)).union(set(list3)).union(set(list4)).union(set(list5)).union(set(list6)).union(set(list7)).union(set(list8)).union(set(list9)).union(set(list10)).union(set(list11)).union(set(list12)).union(set(list13)).union(set(list14)).union(set(list15)).union(set(list16)).union(set(list17))
masterlist1 = list(masterset1)

#------Create Master Protein Fasta-------

findProtSeq(masterlist1, "chat-fastadump1.fasta")

end = time.time()
print(end-start)

4076.746216058731


In [11]:
start = time.time()

#----------Process All datasets----------

"""
c1 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/1-DTAref.txt", "customDB-Peptides/1-Peptides.txt")
c2 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/2-DTAref.txt", "customDB-Peptides/2-Peptides.txt")
c3 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/3-DTAref.txt", "customDB-Peptides/3-Peptides.txt")
c4 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/4-DTAref.txt", "customDB-Peptides/4-Peptides.txt")
c5 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/5-DTAref.txt", "customDB-Peptides/5-Peptides.txt")
c6 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/6-DTAref.txt", "customDB-Peptides/6-Peptides.txt")
c7 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/7-DTAref.txt", "customDB-Peptides/7-Peptides.txt")
c8 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/8-DTAref.txt", "customDB-Peptides/8-Peptides.txt")
c9 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/9-DTAref.txt", "customDB-Peptides/9-Peptides.txt")
c10 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/10-DTAref.txt", "customDB-Peptides/10-Peptides.txt")
c11 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/11-DTAref.txt", "customDB-Peptides/11-Peptides.txt")
c12 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/12-DTAref.txt", "customDB-Peptides/12-Peptides.txt")
c13 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/13-DTAref.txt", "customDB-Peptides/13-Peptides.txt")
c14 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/14-DTAref.txt", "customDB-Peptides/14-Peptides.txt")
c15 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/15-DTAref.txt", "customDB-Peptides/15-Peptides.txt")
c16 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/16-DTAref.txt", "customDB-Peptides/16-Peptides.txt")
c17 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/17-DTAref.txt", "customDB-Peptides/17-Peptides.txt")
"""

c18 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/18-DTAref.txt", "customDB-Peptides/18-Peptides.txt")
c19 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/19-DTAref.txt", "customDB-Peptides/19-Peptides.txt")
c20 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/20-DTAref.txt", "customDB-Peptides/20-Peptides.txt")
c21 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/21-DTAref.txt", "customDB-Peptides/21-Peptides.txt")
c22 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/22-DTAref.txt", "customDB-Peptides/22-Peptides.txt")
c23 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/23-DTAref.txt", "customDB-Peptides/23-Peptides.txt")
c24 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/24-DTAref.txt", "customDB-Peptides/24-Peptides.txt")
c25 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/25-DTAref.txt", "customDB-Peptides/25-Peptides.txt")
c26 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/26-DTAref.txt", "customDB-Peptides/26-Peptides.txt")
c27 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/27-DTAref.txt", "customDB-Peptides/27-Peptides.txt")
c28 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/28-DTAref.txt", "customDB-Peptides/28-Peptides.txt")
c29 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/29-DTAref.txt", "customDB-Peptides/29-Peptides.txt")
c30 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/30-DTAref.txt", "customDB-Peptides/30-Peptides.txt")
c31 = processAll("CDHIT/bby-ref-95.clstr", "customDB-DTASelect/31-DTAref.txt", "customDB-Peptides/31-Peptides.txt")


#----------Combine All datasets----------

masterdict1 = {}
dummylist1 = []
"""
for key1 in sorted(c1.keys()):
    dummylist1.append(c1[key1])
    dummylist1.append(c2[key1])
    dummylist1.append(c3[key1])
    dummylist1.append(c4[key1])
    dummylist1.append(c5[key1])
    dummylist1.append(c6[key1])
    dummylist1.append(c7[key1])
    dummylist1.append(c8[key1])
    dummylist1.append(c9[key1])
    dummylist1.append(c10[key1])
    dummylist1.append(c11[key1])
    dummylist1.append(c12[key1])
    dummylist1.append(c13[key1])
    dummylist1.append(c14[key1])
    dummylist1.append(c15[key1])
    dummylist1.append(c16[key1])
    dummylist1.append(c17[key1])
    masterdict1[key1] = dummylist1
    dummylist1 = []
"""   
for key1 in sorted(c18.keys()):
    dummylist1.append(c18[key1])
    dummylist1.append(c19[key1])
    dummylist1.append(c20[key1])
    dummylist1.append(c21[key1])
    dummylist1.append(c22[key1])
    dummylist1.append(c23[key1])
    dummylist1.append(c24[key1])
    dummylist1.append(c25[key1])
    dummylist1.append(c26[key1])
    dummylist1.append(c27[key1])
    dummylist1.append(c28[key1])
    dummylist1.append(c29[key1])
    dummylist1.append(c30[key1])
    dummylist1.append(c31[key1])
    masterdict1[key1] = dummylist1
    dummylist1 = []

#--------------Output Data--------------

outputfile1 = open("customDB-THdump95.txt", 'w')
tempvar1 = ""
tempvar2 = ""
for key1 in sorted(masterdict1.keys()):
    #if (sum(masterdict1[key1]) != 0):
        tempvar2 = str(masterdict1[key1]).replace('[', "").replace(']', "").replace(", ", ',')
        tempvar1 = "Cluster" + str(key1) + ',' + tempvar2 + '\n'
        outputfile1.write(tempvar1)
outputfile1.close()

end = time.time()
print(end-start)

Single Cluster Assignments: 6702
Multi Cluster Assignments: 798
Single Cluster Assignments: 5445
Multi Cluster Assignments: 653
Single Cluster Assignments: 6504
Multi Cluster Assignments: 696
Single Cluster Assignments: 6106
Multi Cluster Assignments: 729
Single Cluster Assignments: 5688
Multi Cluster Assignments: 765
Single Cluster Assignments: 6289
Multi Cluster Assignments: 729
Single Cluster Assignments: 7073
Multi Cluster Assignments: 775
Single Cluster Assignments: 3879
Multi Cluster Assignments: 556
Single Cluster Assignments: 4591
Multi Cluster Assignments: 690
Single Cluster Assignments: 4875
Multi Cluster Assignments: 753
Single Cluster Assignments: 5232
Multi Cluster Assignments: 661
Single Cluster Assignments: 4363
Multi Cluster Assignments: 578
Single Cluster Assignments: 5170
Multi Cluster Assignments: 714
Single Cluster Assignments: 4485
Multi Cluster Assignments: 615
68.09545683860779
